# <font color='darkblue'>Preface</font>
This <a href='http://localhost/jforum/posts/list/4075.page'>chapter</a> covers 
* Making standard control structures functional 
* Abstracting control structures 
* Abstracting iteration 
* Using the right types

You now have all the types of functions you’ll need. As you saw in the <b><a href='http://localhost/jforum/posts/list/4070.page'>previous chapter</a></b>, these functions don’t require any exceptions to the traditional Java coding rules. Using methods as pure functions (<font color='brown'>a.k.a. functional methods</font>) is perfectly in line with most so-called Java best practices. You haven’t changed the rules or added any exotic constructs. You’ve just added <b>some restrictions about what functional methods can do: they can return a value, and that’s all. They can’t mutate any objects or references in the enclosing scope, nor their arguments. In the first part of this chapter, you’ll learn how to apply the same principles to Java control structures</b>. 

<b>In the second part of this chapter, you’ll learn how to abstract collection operations and control structures to use the power of functions.</b> The last part of the chapter presents techniques that will allow you to get the most out of the type system when handling business problems. 

# <font color='darkblue'>Making standard control structures functional </font>
<b>Control structures are the main building blocks of imperative programming</b>. No imperative Java programmer would believe it’s possible to write programs without using if ... else, switch ... case, and for, while, and do loops. These structures are the essence of imperative programming. But in the following chapters, you’ll learn how to write functional programs with absolutely no control structures. <b>In this section, you’ll be less adventurous—we’ll only look at using the traditional control structures in a more functional style.</b> 

One point you learned in <a href='https://www.safaribooksonline.com/library/view/functional-programming-in/9781617292736/kindle_split_009.html#ch02'><b>chapter 2</b></a> is that purely functional methods can’t do anything but return a value. They can’t mutate an object or reference in the enclosing scope. The value returned by a method can depend only on its arguments, although the method can read data in the enclosing scope. In such a case, the data is considered to be implicit arguments. In imperative programming, control structures define a scope in which they generally do something, which means they have an effect. This effect might be visible only inside the scope of the control structure, or it might be visible in the enclosing scope. The control structures might also access the enclosing scope to read values. The following listing shows a basic example of email validation:
```python
import re

emailPtn = re.compile('^[a-z0-9._%+-]+@[a-z0-9.-]+\\.[a-z]{2,4}$')

def testMail(email):
    if emailPtn.match(email):
        sendVerificationMail(email)
    else:
        logError('Email {} is invalid'.format(email))

def sendVerificationMail(email):
    print('Verification mail is sent to {}'.format(email))

def logError(msg):
    print('Error msg: {}'.format(msg))
```
<b><font color='darkblue'>Listing 3.1. Simple email validation</font> </b>

# <font color='darkblue'>Abstracting control structures </font>
The code in listing 3.1 is purely imperative. You’ll never find such code in functional programming. <b>Although the <font color='blue'>testMail</font> method seems to be a pure effect because it doesn’t return anything, it mixes data processing with effects. This is something you want to avoid, because it results in code that’s impossible to test.</b> Let’s see how you can clean this up. 

<b>The first thing you may want to do is separate computation and effects so you can test the computation result</b>. This could be done imperatively, but I prefer to use a function, as shown in the following listing.
```python
emailPtn = re.compile('^[a-z0-9._%+-]+@[a-z0-9.-]+\\.[a-z]{2,4}$')
# Compose email checker as a external function for testing
emailChecker = lambda e: emailPtn.match(e)

def testMail(email):
    if emailChecker(email):
        sendVerificationMail(email)
    else:
        logError('Email {} is invalid'.format(email))
```
<b><font color='darkblue'>Listing 3.2. Using a function to validate the email </font></b>

Now you can test the data processing part of the program (<font color='brown'>validating the email string</font>) because you’ve clearly separated it from the effects. But you still have many problems. One is that you handle only the case where the string doesn’t validate. But if the string received is None, a <font color='blue'><b>TypeError</b></font>  is thrown. Consider the following example: 
```python
testMail("john.doe@acme.com");
testMail(None);
testMail("paul.smith@acme.com");
```
The third line won’t be executed, even though the email address is valid, because the NPE thrown by the second line kills the thread. It would be better to get a logged message indicating what happened, and to continue processing the next address. Another problem appears if you receive an empty string: 
```python
testMail("");  
```
This won’t cause an error, but the address won’t validate, and the following message will be logged:
> <font color='green'>email is invalid.</font>

The double space (between “email” and “is”) indicates that the string was empty. A specific message would be better, such as this: 
> <font color='green'>email must not be empty.</font>

To handle these problems, we define a class <font color='blue'><b>Result</b></font> to handle this kind of case. So you can write your new version of the program:

In [1]:
import re
from fpu.fp import *

class EmailValidation:
    emailPattern = re.compile("^[a-z0-9._%+-]+@[a-z0-9.-]+\\.[a-z]{2,4}$")
    
    def emailChecker(self, e):
        if e is None:
            return Result.failure('email must not be null')
        elif len(e.strip()) == 0:
            return Result.failure('email must not be empty')
        elif self.emailPattern.match(e):
            return Result.success()
        else:
            return Result.failure("email " + e + " is invalid.")
        
    def logError(self, s):
        print("Error message logged: " + s)
        
    def sendVerificationMail(self, s):
        print("Mail sent to " + s)
        
    def validate(self, e):
        rst = self.emailChecker(e)
        if isinstance(rst, Success):
            self.sendVerificationMail(e)
        else:
            self.logError(rst.message)
            
ev = EmailValidation()
ev.validate('this.is@my.email')
ev.validate('test@abc.com')
ev.validate(None)
ev.validate('')

Error message logged: email this.is@my.email is invalid.
Mail sent to test@abc.com
Error message logged: email must not be null
Error message logged: email must not be empty


But this still isn’t satisfactory. Using <a href='https://docs.python.org/2/library/functions.html#isinstance'>isinstance</a> to determine whether the result is a success is ugly. <b>But worse than this is the fact that you have some program logic in the <font color='blue'>validate</font> method that can’t be tested. This is because the method is an effect, which means it doesn’t return a value but mutates the outside world. </b> Is there a way to fix this? Yes. Instead of sending an email or logging a message, you could return a small program that does the same thing. Instead of executing:
```python
sendVerificationMail(s)  
```

and
```python
logError(((Result.Failure) result).getMessage());  
```

you could return instructions that, when executed, will produce the same results. Thanks to lambdas, you can do this easily. Here we provide class <b><font color='blue'>Supplier</font></b> to store the execution step. So you can easily change your program, as shown in the following listing: 

In [2]:
class EmailValidation2:
    emailPattern = re.compile("^[a-z0-9._%+-]+@[a-z0-9.-]+\\.[a-z]{2,4}$")
    
    def emailChecker(self, e):
        if e is None:
            return Result.failure('email must not be null')
        elif len(e.strip()) == 0:
            return Result.failure('email must not be empty')
        elif self.emailPattern.match(e):
            return Result.success()
        else:
            return Result.failure("email " + e + " is invalid.")
        
    def logError(self, s):
        print("Error message logged: " + s)
        
    def sendVerificationMail(self, s):
        print("Mail sent to " + s)
        
    # (2)
    def validate(self, e):
        rst = self.emailChecker(e)
        if isinstance(rst, Success):
            return Supplier(self.sendVerificationMail, e)
        else:
            return Supplier(self.logError, rst.message)
        
ev = EmailValidation2()
# (1)
ev.validate('this.is@my.email')()
ev.validate('test@abc.com')()
ev.validate(None)()
ev.validate('')()    

Error message logged: email this.is@my.email is invalid.
Mail sent to test@abc.com
Error message logged: email must not be null
Error message logged: email must not be empty


The validate method (2) now returns <b><font color='blue'>Supplier</font></b> instead of void. It no longer has any side effect, and it’s a pure function. When an <b><font color='blue'>Supplier</font></b> is returned (1), it can be executed by calling it directly. Note that the <b><font color='blue'>Supplier</font></b> could also be passed to other methods or stored away to be executed later. In particular, it could be put in a data structure and executed in sequence after all computations are done. <b>This allows you to separate the functional part of the program from the part that mutates the environment</b>. 

## <font color='green'>Cleaning up the code </font>
Your <font color='blue'>validate</font> method is now functional, but it’s dirty. Using the <a href='https://docs.python.org/2/library/functions.html#isinstance'>isinstance</a> operator is almost always an indication of bad code. Another problem is that reusability is low. When the <font color='blue'>validate</font> method returns a value, you have no choice besides executing it or not. What if you want to reuse the <font color='blue'>validate</font> part but produce a different effect?

The <font color='blue'>validate</font> method shouldn’t have a dependency on <font color='blue'>sendVerificationMail</font> or <font color='blue'>logError</font>. <b>It should only return a result expressing whether the email is valid, and you should be able to choose whatever effects you need for success or failure</b>. Or you might prefer not to apply the effect but to compose the result with some other processing. Let's try to decouple the <font color='blue'>validate</font> from the effects applied:

In [3]:
# Listing 3.7. A cleaner version of the program 
class EmailValidation3:
    emailPattern = re.compile("^[a-z0-9._%+-]+@[a-z0-9.-]+\\.[a-z]{2,4}$")
    class SuccessEft(Effect):
        def call(self, s):
            print("Mail sent to {}".format(s))
            
    class FailureEft(Effect): 
        def call(self, s):
            print("Error message logged: {}".format(s))
    
    def emailChecker(self, e):
        if e is None:
            return Result.failure('email must not be None')
        elif len(e.strip()) == 0:
            return Result.failure('email must not be empty')
        elif self.emailPattern.match(e):
            return Result.success(e)
        else:
            return Result.failure("email " + e + " is invalid.")
        

ev = EmailValidation3()
st = EmailValidation3.SuccessEft()
ft = EmailValidation3.FailureEft()
# (1)
ev.emailChecker('this.is@my.email').bind(st, ft)  # .bind(SuccessEffect, FailureEffect)
ev.emailChecker('test@abc.com').bind(st, ft)
ev.emailChecker(None).bind(st, ft)
ev.emailChecker('').bind(st, ft)

Error message logged: email this.is@my.email is invalid.
Mail sent to test@abc.com
Error message logged: email must not be None
Error message logged: email must not be empty


The <font color='blue'>validate</font> method has been removed, and two <font color='blue'>Effect</font> instances are now defined : one for success and one for failure. These two effects are bound to the result of the <font color='blue'>emailChecker</font> function. 

## <font color='green'>An alternative to if ... else</font>
You may wonder whether it’s possible to completely remove conditional structures or operators. Can you write a program without any of these constructs? This may seem impossible, because many programmers have learned that decision-making is the basic building block of programming. <b>But decision-making is an imperative programming notion. It’s the notion of examining a value and deciding what to do next based on this observation. In functional programming, there’s no “what to do next” question, but only functions returning values.</b> The most basic if structure may be seen as the implementation of a function: 
```java
if x > 0:  
  return x  
else  
  return -x  
```
This is a function of <i>x</i>. It returns the absolute value of <i>x</i>. You could write this function as follows: 
```python
abs = lambda x: x if x > 0 else -x
```
The difference with a function such as:
```python
square = lambda x: x * x
```
is that you have two implementations of the function and have to choose between the two depending on the value of the argument. This isn’t a big problem, but what if you had many possible implementations? You’d end up with as many embedded if ... else structures as you have in listing 3.7.

Here we will use <b><font color='blue'>Case</font></b> class representing a condition and corresponding result. The condition will be represented by a <b><font color='blue'>Supplier</font></b>, where <b><font color='blue'>Supplier</font></b> is a functional interface such as this:
```python
class Supplier:
    def __init__(self, func, *args):
        self.func = func
        self.args = args

    def __call__(self):
        return self.func(*self.args)
```
The result corresponding to the condition will be represented by a <b><font color='blue'>Supplier</font></b>. To hold both, you can use a <b>Tuple<Supplier<Boolean>, Supplier<Result<T>>></b>. The <b><font color='blue'>Case</font></b> class should define three methods: 
```python
    @staticmethod
    def mcase(cond, rst):
        r'''
        Conditional clause
        '''
        ...

    @staticmethod
    def default(rst):
        r'''
        Default clause
        '''
        ...

    @staticmethod
    def match(dftCase, *matches):
        r'''
        Perform matching clause
        '''
        ...    
```
The first <font color='blue'>mcase</font> method defines a normal case, with a condition and a resulting value. The second <font color='blue'>default</font> method defines a default case. The third method, <font color='blue'>match</font>, selects a case. Because this method uses a vararg, the default case (_dftCase_) is to be put first, but will be the last to be used!

Now you can greatly simplify the code of your email validation application. As you can see in the following listing, it contains absolutely no control structures:

In [4]:
# Listing 3.9. The email validation application with no control structures
from fpu.fp import *

class EmailValidation4:
    emailPattern = re.compile("^[a-z0-9._%+-]+@[a-z0-9.-]+\\.[a-z]{2,4}$")
    class SuccessEft(Effect):
        def call(self, s):
            print("Mail sent to {}".format(s))
            
    class FailureEft(Effect): 
        def call(self, s):
            print("Error message logged: {}".format(s))
            
    def emailChecker(self, e):      
        # if/else is gone
        return Case.match(
                          Case.default(Result.failure("email " + str(e) + " is invalid.")),                                              # Default Case
                          Case.mcase(Supplier(lambda e: e is None, e), 
                                     Result.failure('email must not be None')),    
                          Case.mcase(Supplier(lambda e: len(e.strip()) == 0, e), 
                                     Result.failure('email must not be empty')),
                          Case.mcase(Supplier(lambda e: self.emailPattern.match(e)!=None, e), 
                                     Result.success(e))
                          )
    
ev = EmailValidation4()
st = EmailValidation3.SuccessEft()
ft = EmailValidation3.FailureEft()
# (1)
ev.emailChecker('this.is@my.email').bind(st, ft)  # .bind(SuccessEffect, FailureEffect)
ev.emailChecker('test@abc.com').bind(st, ft)
ev.emailChecker(None).bind(st, ft)
ev.emailChecker('').bind(st, ft)    

Error message logged: email this.is@my.email is invalid.
Mail sent to test@abc.com
Error message logged: email must not be None
Error message logged: email must not be empty


# <font color='darkblue'>Abstracting iteration </font>
Getting back to loops, their main use is to iterate over all elements of a list, as follows: 
```python
for mail in mailList:
    # Do something with email
    pass

for i in range(len(mailList)):
    mail = mailList[i]
    # Do something with email
```
What’s important is what’s done inside the loop, so why should you have to write the loops again and again? Why can’t you just say what you want done and have it be done without messing with the control structures, the conditions, and the indexes? 

Inside a loop, you might want to do several things: 
* Transform each element into something else 
* Aggregate elements into a single result 
* Remove some elements according to a condition on the elements 
* Remove some elements according to an external condition 
* Group elements according to certain criteria

Various operations for which looping is needed can be applied to collections, such as concatenating, zipping, or unzipping. (<font color='brown'>Zipping means taking elements from two lists and creating a list of tuples. Unzipping is the inverse operation.</font>) All these operations could be abstracted. In chapter 5, you’ll create functional data structures implementing all these abstractions. For now, you’ll develop a library of these abstractions that you can apply to legacy Python collections. 

## <font color='green'>Abstracting an operation on lists with mapping</font>
Mapping, when applied to collections, means applying a transformation to each element of the collection. Here’s how it’s generally done in traditional imperative programming: 
```python
datas = [1, 2, 3, 4, 5]
new_datas = []
for e in datas:
    new_datas.append(e * 1.2)
```

In this example, an operation is applied to each element of an Integer list (integerList) to increase it by 20%. The result of the operation is a double, so it’s put in a new list that’s created before the start of the loop. Although simple, this program raises some interesting questions. The first point is that you could separate the iteration from the calculation. The following example does this with a method:
```python
def addTwentyPercent(e):
    return e * 1.2

datas = [1, 2, 3, 4, 5]
new_datas = []
for e in datas:
    new_datas.append(addTwentyPercent(e))
```

This allows you to reuse the calculation, but it doesn’t allow you to reuse the loop. Fortunately, Python has built-in function <a href='https://docs.python.org/3/library/functions.html#map'>map</a> to carry out the loop operation on every element of list. For example:

In [5]:
def addTwentyPercent(e):
    return e * 1.2

datas = [1, 2, 3, 4, 5]
print('datas=%s' % datas)
print('datas increased by 20%={}'.format(map(addTwentyPercent, datas)))

datas=[1, 2, 3, 4, 5]
datas increased by 20%=[1.2, 2.4, 3.5999999999999996, 4.8, 6.0]


## <font color='green'>Reducing and folding lists </font>
<b>List folding transforms a list into a single value by using a specific operation</b>. The resulting value may be of any type—it doesn’t have to be of the same type as the elements of the list. <b>Folding to a result that’s the same type as the list elements is a specific case called reducing. Computing the sum of the elements of a list of integers is a simple case of reducing</b>. You can fold a list in two directions, from left to right or from right to left, depending on the operation used: 
* If the operation is commutative, both ways of folding are equivalent. 
* If the operation isn’t commutative, the two ways of folding give different results.

<b>Folding needs a starting value, which is the neutral element, or identity element, for the operation. This element is used as the starting value of the accumulator</b>. When the computation is complete, the accumulator contains the result. Reducing, on the other hand, can be done without a starting element, with the condition that the list isn’t empty, because the first (<font color='brown'>or last</font>) element will be used as the starting element. 
<br/>
### Reducing lists of numbers with addition
Suppose you have a list, (1, 2, 3, 4), and you want to compute the sum of the elements. The first way to do it is to put the accumulator on the left side of the operand: 
```
(((0 + 1) + 2) + 3) + 4 = 10
```

You could also proceed from the other side: 
```
1 + (2 + (3 + (4 + 0))) = 10
```
The results are identical. You could do the same thing with multiplication, but you’d have to use the identity element 1 as the starting value of the accumulator. 

### Folding lists of characters into strings 
Let’s now do the same thing with a different operation applied to a list of characters, ('a', 'b', 'c'). First, let’s fold from the left: 
```
(("" + 'a') + 'b') + 'c' = "abc"
```
Let’s now try the same thing from the right: 
```
'a' + ('b' + ('c' + "")) = "abc"  
```
The first fold is called a left fold, which means that the accumulator is on the left side of the operation. When the accumulator is on the right side, it’s a right fold. 

### Understanding the relationship between left and right folds
You might say that folding right can be defined in terms of folding left. Let’s rewrite the right-folding operation by using a different form, called <b><font color='darkblue'>corecursion</font></b>: 
```
((0 + 3) + 2) + 1 = 6
```
In recursion as well as corecursion, evaluation of one step is dependent on the previous step. But a recursive definition starts with the last step and defines its relationship with the preceding one. In order to be able to conclude, it also has to define the base step. Corecursion, on the other hand, starts from the first step and defines its relationship to the next one. There’s no need for a base step, because it’s also the first step. <b>From this, it seems that right-folding a list is equivalent to left-folding the list after having reversed the order of the elements</b>. 

<font color='red'>But wait. Addition is a commutative operation. If you use a noncommutative operation, you must change the operation as well.</font> If you don’t, you could end up with two different situations, depending on the types. If the operation has operands of different types, if won’t compile. On the other hand, if the operation has operands of the same types but it isn’t commutative, you’ll get a wrong result with no error. So foldLeft and foldRight have the following relationship, where operation1 and operation2 give the same results with the same operands in reverse order:
```
foldLeft(list, acc, x -> y -> operation1)  
```
is equivalent to 
```
foldRight(reverse(list), acc, y -> x -> operation2) 
```

### Left-folding example 
The operation you just defined was named fold because folding left or right for integer addition or multiplication gives the same result. But if you want to use other functions, or <b>if you want to make the folding method generic, you must distinguish between right and left folds</b>. Let's generalize the fold method to foldLeft so that it can be used to apply a left fold to a list of elements of arbitrary types (<font color='brown'>Exercise 3.6</font>). To test that the method is correct, apply it to the following parameters, 

In [6]:
from fpu import fp
alist = [1, 2, 3, 4, 5]
idy = '0'  # Identity
fp.CollectionUtils.foldLeft(alist, idy, lambda x,y: "({} + {})".format(x, y))

'(((((0 + 1) + 2) + 3) + 4) + 5)'

### Right-folding example 
As you saw previously, <b>folding left is a corecursive operation, so implementing it through an imperative loop is easy. On the other hand, folding right is a recursive operation</b>. Now let's take a look at right folding example:

In [7]:
fp.CollectionUtils.foldRight(alist, idy, lambda x,y: "({} + {})".format(x, y))

'(1 + (2 + (3 + (4 + (5 + 0)))))'

### Reversing a list
Reversing a list is sometimes useful, although this operation is generally not optimal in terms of performance. Finding other solutions that don’t require reversing a list is preferable, but not always possible. Defining a reverse method with an imperative implementation is easy by iterating backward over the list. You must be careful, though, not to mess with the indexes: 

In [8]:
import random

alist = []
for i in range(10):
    alist.append(random.randint(1, 1000))

def reverse(alist):
    r'''
    Imperative implementation for reversing list
    '''
    nlist = []
    for i in range(len(alist)-1, -1, -1):
        nlist.append(alist[i])
        
    return nlist
    
print(alist)        
print(reverse(alist))

[78, 596, 638, 570, 291, 897, 814, 803, 214, 336]
[336, 214, 803, 814, 897, 291, 570, 638, 596, 78]


The code above is straight forward and easy to understand. However the code is not neat and cute enough:

In [9]:
fp.CollectionUtils.foldRight(alist, [], lambda e, nlist: nlist + [e])

[336, 214, 803, 814, 897, 291, 570, 638, 596, 78]

<b>Part of the beauty of functional programming is in finding every small element that can be abstracted and reused</b>. After you get used to this way of thinking, you’ll start seeing patterns everywhere and you’ll want to abstract them. You could define lots of other useful functions by composing the basic list functions you just wrote. But we’ll delay their study until <a href='https://www.safaribooksonline.com/library/view/functional-programming-in/9781617292736/kindle_split_012.html#ch05'>chapter 5</a>

## <font color='green'>Composing mappings and mapping compositions</font>
It isn’t unusual to apply several transformations to list elements. Imagine you have a list of prices, and you want to apply a 9% tax to all, and then add a fixed charge of $3.50 for shipping. You can do this by composing two mappings: 

In [10]:
prices = [10.10, 23.45, 32.07, 9.23]
pricesIncludingTax = map(lambda e: e * 1.09, prices)
pricesIncludingShipping = map(lambda e: e + 3.5, pricesIncludingTax)
print(pricesIncludingShipping)

[14.509, 29.0605, 38.456300000000006, 13.5607]


It works but it isn’t efficient, because mapping is applied twice. You could obtain the same result with this

In [11]:
map(lambda e: e + 3.5,  map(lambda e: e * 1.09, prices))

[14.509, 29.0605, 38.456300000000006, 13.5607]

But this is still mapping twice. <b>A much better solution is to compose the functions instead of composing mappings, or, in other words, to map the composition instead of composing mappings</b>: 

In [12]:
f = lambda e: e + 3.5
g = lambda e: e * 1.09
uprice = fp.compose2(f, g)  # f(g(x)) 
map(uprice, prices)

[14.509, 29.0605, 38.456300000000006, 13.5607]

## <font color='green'>Applying effects to lists</font>
 In a real situation, you’d probably apply more-sophisticated effects to each element of the list. You could, for example, print each price after formatting it to display only two decimal digits. This could be done through iteration:

In [13]:
for price in pricesIncludingShipping:
    print("{:.02f}".format(price))

14.51
29.06
38.46
13.56


But once again, you’re mixing actions that could be abstracted. <b>Iteration can be abstracted exactly as you did for mapping, and the effect applied to each element could be abstracted into something resembling a function, but with a side effect and no return value</b>. This is exactly what the <font color='violet'><b>Effect</b></font> you used in below user case:

In [14]:
class PrintEfft(fp.Effect):
    def call(self, value):
        print('{:.02f}'.format(value))

fp.CollectionUtils.forEach(pricesIncludingShipping, PrintEfft())

14.51
29.06
38.46
13.56


This seems to be much more code, but the <font color='blue'><b>PrintEfft</b></font> class and the <font color='blue'>forEach</font> method can be written once and reused, so you can test each of them in isolation. Your business code is reduced to only one line. 

## <font color='green'>Approaching functional output</font>
With the <font color='blue'>forEach</font> method, you can somewhat abstract side effects. You abstracted effect application so it can be isolated, but you could go much further. With the <font color='blue'>forEach</font> method, one single effect is applied to each element of the list. It would be nice to be able to compose these effects into a single one. Think about it as a fold resulting in a single effect. If you could do this, your program could be a fully functional one with absolutely no side effects. <b>It would produce a new program, with no control structures but a single list of effects that would be applied one after the other</b>. Let’s do this! 

In [15]:
def printEffect(i, e):
    print('{:.02f}'.format(e))
    return e

def gt20(e):
    if e > 20:
        print('Great than 20!')
    
fun = Fun(printEffect).andThen(gt20)    

sup = fp.Supplier(fp.CollectionUtils.foldLeft, pricesIncludingShipping, None, fun)  # Portable function object
sup()

14.51
29.06
Great than 20!
38.46
Great than 20!
13.56


(None,)

## <font color='green'>Building corecursive lists</font>
<b>One thing programmers do again and again is build corecursive lists, and most of these are lists of integers.</b> consider the following example:
```python
for i in range(limit):
    pass # Some processing
```

This code is a composition of two abstractions: a corecursive list and some processing. <b>The corecursive list is a list of integers from 0 (included) to limit (excluded)</b>. As we’ve already noted, functional programming is, among other things, about pushing abstraction to the limit. So let’s abstract the construction of this corecursive list. As I mentioned earlier, <b>corecursive means that each element can be constructed by applying a function to the previous element, starting from the first one.</b> This is what distinguishes corecursive from recursive constructs. (In recursive constructs, each element is a function of the previous one, starting with the last one.) We’ll come back to this difference in <b><a href='https://www.safaribooksonline.com/library/view/functional-programming-in/9781617292736/kindle_split_011.html#ch04'>chapter 4</a></b>, but for now, this means that corecursive lists are easy to construct.

Let's say if we want to generate a <b><a href='https://en.wikipedia.org/wiki/Fibonacci_number'>Fibonacci number</a></b> series with length as 10:

In [16]:
fibon_series_10 = [1, 1]

# loop approach
#while len(fibon_series_10) < 10:
#    fibon_series_10.append(fibon_series_10[-1] + fibon_series_10[-2])

# Recursive
def fib_recv(series, length):
    if len(series) == length:
        return series
    
    if len(series) < 2:
        series.append(1)
        return fib_recv(series, length)
    else:
        series.append(series[-1])

print(fibon_series_10)

[1, 1]


But with FP, the code is more easy to read:

In [17]:
def fib_unfold(t):
    if len(t) == 0:
        t.append(1)        
    elif len(t) == 1:
        t.append(1)        
    else:
        t.append(t[-1] + t[-2])
        
    return t

fibon_series_10 = []
fp.CollectionUtils.unfold(fibon_series_10, fib_unfold, lambda t: len(t) < 10)
print(fibon_series_10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


Above sample code introduced the functional call <font color='blue'>unfold</font> which takes three parameters:
```python
    @staticmethod
    def unfold(seed, f, p):
        r'''
        @param seed:
            The input argument of second and third callable object
        @param f(callable):
            Generate the next value for collection
        @param p(callable):
            Callable to terminate the process by returning True
        '''
```

Sometimes, you want to apply function to every element in a list, you can use <font color='blue'>forEach</font>:

In [19]:
def mult10(e):
    print('{}'.format(e * 10))
    
fp.CollectionUtils.forEach([1, 2, 4], mult10)

10
20
40
